# OpEn Rust Example Test

This example solves a simple optimisation problem by using [OpEn's rust interface](https://alphaville.github.io/optimization-engine/docs/openrust-basic).

## Problem Formulation

**OpEn** can solve problems of the form:

$$\min_{u \in \mathbb{R}^{n_u}} f(u,p)$$


subject to

$$u \in U(p)$$



where 
- $f$ is a $C^{1,1}$ function (continously differentiable with Lipschitz-continuous gradient); 
- $U$ is a set where we may project.





## Example 1: panoc_ex1.rs


The following is "panoc_ex1.rs" example in [here](https://alphaville.github.io/optimization-engine/docs/openrust-basic).


First, we need to import "optimization_engine"
- In your local PC, it should be also declared in "Cargo.toml".
- In this jupyter notebook, we need to have "extern crate" as follows. 


In [2]:
extern crate optimization_engine;
use optimization_engine::{constraints::*, panoc::*, *};

Then, we need to define the objective function that we are going to optimise, and its gradient function. 

In this example, we use Rosenbrock function. 

$$f(u,a,b) = (a - u_1)^2 + b(u_2 - u_1^2)^2$$

with gradient

$$\frac{df(u,a,b)}{du} = [2(u_1 - a) - 4 b u_1 (u_2 - u_1^2), 2 b (u_2 - u_1^2) ]^{T}$$



In [3]:
fn rosenbrock_cost(a: f64, b: f64, u: &[f64]) -> f64 {
    (a - u[0]).powi(2) + b * (u[1] - u[0].powi(2)).powi(2)
}


In [4]:
fn rosenbrock_grad(a: f64, b: f64, u: &[f64], grad: &mut [f64]) {
    grad[0] = 2.0 * u[0] - 2.0 * a - 4.0 * b * u[0] * (-u[0].powi(2) + u[1]);
    grad[1] = b * (-2.0 * u[0].powi(2) + 2.0 * u[1]);
}

Now, the main function should look like this:

In [5]:
fn main() {
    /* USER PARAMETERS */
    let tolerance = 1e-14;
    let a = 1.0;
    let b = 200.0;
    let problem_size = 2;
    let lbfgs_memory_size = 10;
    let max_iters = 80;
    let mut u = [-1.5, 0.9];
    let radius = 1.0;

    // define the cost function and its gradient
    let df = |u: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
        if a < 0.0 || b < 0.0 {
            Err(SolverError::Cost)
        } else {
            rosenbrock_grad(a, b, u, grad);
            Ok(())
        }
    };

    let f = |u: &[f64], c: &mut f64| -> Result<(), SolverError> {
        if a < 0.0 || b < 0.0 {
            Err(SolverError::Cost)
        } else {
            *c = rosenbrock_cost(a, b, u);
            Ok(())
        }
    };

    // define the constraints
    let bounds = Ball2::new(None, radius);

    /* PROBLEM STATEMENT */
    let problem = Problem::new(&bounds, df, f);
    let mut panoc_cache = PANOCCache::new(problem_size, tolerance, lbfgs_memory_size);
    let mut panoc = PANOCOptimizer::new(problem, &mut panoc_cache).with_max_iter(max_iters);

    // Invoke the solver
    let status = panoc.solve(&mut u);

    println!("Panoc status: {:#?}", status);
    println!("Panoc solution: {:#?}", u);
}

Let's excute it.

In [6]:
main();


Panoc status: Ok(
    SolverStatus {
        exit_status: Converged,
        num_iter: 58,
        solve_time: 23.286µs,
        fpr_norm: 0.000000000000002826166425630795,
        cost_value: 0.04570299540701696,
    },
)
Panoc solution: [
    0.7862834045323073,
    0.6178660111683472,
]


Note that, in your local PC, we can run the code by "cargo build" and "cargo run". 


## Example 2:

Minimise $$f(x,p) = (x - p_1)^2$$ 

subject to $$x \le p_2$$

where $p \in \mathbb{R}$.

In [7]:
fn example_two_cost(p: &[f64], u: &[f64]) -> f64 {
    (u[0] - p[0]).powi(2)
}

Its gradient is 

$$ df(x,p) = 2(x-p_1) $$

In [8]:
fn example_two_grad(p: &[f64], u: &[f64], grad: &mut [f64]) {
    grad[0] = 2.0 * (u[0] - p[0]);
}

Now, the main is

In [9]:
fn main_example_two(_p: &[f64]) {
    /* USER PARAMETERS */
    let tolerance = 1e-14;
    let p = _p;
    let problem_size = 2;
    let lbfgs_memory_size = 10;
    let max_iters = 80;
    let mut u = [-1.5, 0.9];
    
    let mut upper_bound: Vec<f64> = Vec::new();
    upper_bound.push(p[1]);
    // upper_bound.push(0.0);

    // define the cost function and its gradient
    let df = |u: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
        example_two_grad(&p, u, grad);
        Ok(())
    };

    let f = |u: &[f64], c: &mut f64| -> Result<(), SolverError> {
        *c = example_two_cost(&p, u);
        Ok(())
    };

    // define the constraints
    let bounds = Rectangle::new(None, Option::from(upper_bound.as_slice()));
    //let bounds = Ball2::new(None, upper_bound);

    /* PROBLEM STATEMENT */
    let problem = Problem::new(&bounds, df, f);
    let mut panoc_cache = PANOCCache::new(problem_size, tolerance, lbfgs_memory_size);
    let mut panoc = PANOCOptimizer::new(problem, &mut panoc_cache).with_max_iter(max_iters);

    // Invoke the solver
    let status = panoc.solve(&mut u);

    println!("Panoc status: {:#?}", status);
    println!("Panoc solution: {:#?}", u);
}

#### Result

##### For the case $p_1 = 10$, $p_2 = 5$:

In [10]:
main_example_two(&[10.0,5.0]);

Panoc status: Ok(
    SolverStatus {
        exit_status: Converged,
        num_iter: 1,
        solve_time: 1.399µs,
        fpr_norm: 0.0,
        cost_value: 25.0,
    },
)
Panoc solution: [
    5.0,
    0.9000009,
]


Note that the second solution element is dummy, as I used "u" in "main_example_two" as a vector. 

##### For the case $p_1 = -10, p_2 = 5$:

In [11]:
main_example_two(&[-10.0,5.0]);

Panoc status: Ok(
    SolverStatus {
        exit_status: Converged,
        num_iter: 3,
        solve_time: 3.489µs,
        fpr_norm: 0.0000000000000017763568394002505,
        cost_value: 0.0000000000000000000000000000031554436208840472,
    },
)
Panoc solution: [
    -10.0,
    0.9000009,
]


The above results are the same as those from **the panelty method Python interface** in [here](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_python/OpEn_Python_Panelty.ipynb).

### Comparison with Python Interface: Rust is much faster

Using rust seems much faster: rust solves this example problem in 3 micro seconds, whereas the python interface solves it in 50 ms.  